# 2. Incoming Event Handler
  --------------------------------------------------------------------

Handle incoming events and write them to an output V3IO Stream `incoming-events-stream`.
The received data is partitioned by `user_id`.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

The rest of the notebooks rely on the output stream of this notebook. Therefore, one can change the input data without affecting the rest of the workflow.

## Create and Test a Local Function 
Import nuclio SDK and magics, <b>do not remove the cell and comment !!!</b>

In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
%run config.py

In [ ]:
print(PARTITION_ATTR)

In [ ]:
import os
MDWS_EH_OUTPUT_STREAM_PATH = STREAM_CONFIGS['incoming-events-stream']['path']
input_stream_url = path.join(WEB_API_USERS, STREAM_CONFIGS['generated-stream']['path']) + "@incomingeventhandler"

In [ ]:
# nuclio: start-code

#### Functions imports

In [ ]:
import v3io.dataplane

<b>Specify function dependencies and configuration<b>

In [ ]:
%nuclio cmd -c pip install v3io

In [ ]:
%%nuclio config
spec.build.baseImage = "mlrun/ml-models"

## Function code

In [ ]:
import os
import json
def init_context(context):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    CONTAINER = os.getenv('CONTAINER')
    OUTPUT_STREAM_PATH = os.getenv('MDWS_EH_OUTPUT_STREAM_PATH')
    PARTITION_ATTR = os.getenv('MDWS_PARTITION_ATTR')
    WEB_API = os.getenv('WEB_API')
    v3io_client = v3io.dataplane.Client(endpoint=WEB_API, access_key=V3IO_ACCESS_KEY)

    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'partition_attr', PARTITION_ATTR)
    setattr(context, 'container', CONTAINER)
    setattr(context, 'output_stream_path', OUTPUT_STREAM_PATH)


def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
        
    partition_key = event_dict.get(context.partition_attr)
    record = event_to_record(event_dict, partition_key)
    
    print("saasasas" + context.output_stream_path)
    resp = context.v3io_client.put_records(container=context.container, 
                                   path=context.output_stream_path, 
                                   records=[record], 
                                   raise_for_status=v3io.dataplane.RaiseForStatus.never)
    
    context.logger.info_with('Sent event to stream', 
                             record=record,
                             response_status=resp.status_code, 
                             response_body=resp.body.decode('utf-8'))
    
    return resp.status_code


def event_to_record(event_dict, partition_key):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(partition_key)}

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [ ]:
# nuclio: end-code
# marks the end of a code section

## Test locally

In [ ]:
event = nuclio.Event(body=b'{"user_id" : 111111 , "event_type": "spin"}')
init_context(context)
handler(context, event)

# MLRUN

In [ ]:
%store -r project_name
%store -r project_path
%store -r artifact_path
%store -r project

In [ ]:
from mlrun import code_to_function

gen_func = code_to_function(name='incoming', kind = 'nuclio')
project.set_function(gen_func)
incoming_event_handler = project.func('incoming')
incoming_event_handler.set_envs({'WEB_API' : WEB_API, 'V3IO_ACCESS_KEY': V3IO_ACCESS_KEY, 'CONTAINER': "users", 'MDWS_EH_OUTPUT_STREAM_PATH':MDWS_EH_OUTPUT_STREAM_PATH, 'MDWS_PARTITION_ATTR':PARTITION_ATTR})
incoming_event_handler.add_trigger('incoming', nuclio.triggers.V3IOStreamTrigger(url=input_stream_url, access_key=V3IO_ACCESS_KEY,maxWorkers=10))


In [ ]:
#Build image
incoming_event_handler.deploy()